In [4]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

In [29]:
def replace_html_entities(html):
    html = html.replace('&lt;', '<')
    html = html.replace('&gt;', '>')
    return html

In [38]:
def get_touch_html(match_id):   
    chrome_options = Options()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    driver.get(f"https://www.ligaprofesional.ar/ficha-partido?competition=384&season=2024&match={match_id}")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    mapa_de_toques_tab = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//ul[@class='Opta-Cf']//a[text()='Mapa de toques']"))
    )
    mapa_de_toques_tab.click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".Opta-Teamsheet-Button")))

    home_dropdown_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".Opta-Teamsheet-Button.Opta-Team-Left"))
    )
    home_dropdown_button.click()
    home_team_box = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".Opta-Teamsheet.Opta-Home dt.Opta-Team"))
    )
    driver.execute_script("arguments[0].click();", home_team_box)
    home_dropdown_button.click()

    away_dropdown_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".Opta-Teamsheet-Button.Opta-Team-Right"))
    )
    away_dropdown_button.click()
    away_team_box = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".Opta-Teamsheet.Opta-Away dt.Opta-Team"))
    )
    driver.execute_script("arguments[0].click();", away_team_box)
    away_dropdown_button.click()
    
    optas_touch_sections = driver.find_elements(By.CSS_SELECTOR, ".Opta-touch")
    all_optas_touch_html = ""
    for section in optas_touch_sections:
        all_optas_touch_html += section.get_attribute('outerHTML') + "\n"
     
    home_team_element = driver.find_element(By.CSS_SELECTOR, ".Opta-Team.Opta-Home.Opta-Team-Left.Opta-TeamName")
    away_team_element = driver.find_element(By.CSS_SELECTOR, ".Opta-Team.Opta-Away.Opta-Team-Right.Opta-TeamName")
    
    home_team_html = home_team_element.get_attribute('outerHTML')
    away_team_html = away_team_element.get_attribute('outerHTML')
    
    full_html_content = home_team_html + "\n" + away_team_html + "\n" + all_optas_touch_html
    full_html_content = replace_html_entities(full_html_content)
    
    with open(f"{match_id}_touches.html", "w", encoding="utf-8") as file:
        file.write(full_html_content)

In [45]:
match_id=2420227
get_touch_html(match_id)

In [40]:
with open(f'{match_id}_touches.html', 'r', encoding='utf-8') as file:
    file = file.read()

In [43]:
def scrape_touches(file_name):
    soup = BeautifulSoup(file_name, 'html.parser')
    sections = soup.find_all('circle', class_=lambda x: x and 'Opta-touch' in x and 'Opta-noclick' in x)
    data = []
    
    home_team_tag = soup.find('td', class_=lambda x: x and 'Opta-Team' in x and 'Opta-Home' in x 
                                                     and 'Opta-Team-Left' in x and 'Opta-TeamName' in x)
    home_team = home_team_tag.text.strip().split('\n')[0].strip()

    away_team_tag = soup.find('td', class_=lambda x: x and 'Opta-Team' in x and 'Opta-Away' in x 
                                                     and 'Opta-Team-Right' in x and 'Opta-TeamName' in x)
    away_team = away_team_tag.text.strip().split('\n')[0].strip()

    for section in sections:
        team = away_team if 'Opta-away' in section['class'] else home_team
        cx = section['cx']
        cy = section['cy']

        text_content = section.find('text', class_='Opta-JS-Tip').decode_contents()
        inner_soup = BeautifulSoup(text_content, 'html.parser')
        
        name = inner_soup.find('p').text
        desc = inner_soup.find('span', class_='Opta-Tooltip-Key').text
        minute = inner_soup.find('span', class_='Opta-Tooltip-Value').text.replace('&lrm;', '')
        
        data.append({'team': team, 'player': name, 'x': cx, 'y': cy, 'desc': desc, 'minute': minute})
    
    return pd.DataFrame(data)

In [44]:
scrape_touches(file)

,team,player,x,y,desc,minute
0,Argentinos Juniors,Damián Batallini,398.4,262,Saque inicial,0'
1,Argentinos Juniors,Ariel Gamarra,504,287.676,Pase,1'
2,Argentinos Juniors,Sebastián Prieto,536.8,460.072,Pase,1'
3,Argentinos Juniors,Tobías Palacio,602.4,364.17999999999995,Pase,1'
4,Argentinos Juniors,Francisco Álvarez,568,83.84,Pase,1'
...,...,...,...,...,...,...
1185,Vélez Sarsfield,Christian Ordoñez,588.8,415.00800000000004,Foul,90+6'
1186,Vélez Sarsfield,Lenny Lobato,614.4,408.196,Free kick,90+7'
1187,Vélez Sarsfield,Thiago Vecino,733.6,259.904,Tiro atajado,90+7'
1188,Argentinos Juniors,Diego Rodríguez,785.6,257.284,Save,90+7'
